Copyright © 2020, SAS Institute Inc., Cary, NC, USA.  All Rights Reserved. 
SPDX-License-Identifier: Apache-2.0

# Configure an Azure Container Publishing Destination

In order to create an Azure destination, you must complete the following steps:

1. Find a valid domain with Azure credentials.
2. Create credentials for a specific user or group and define a credential domain.
3. Submit an API post to create an Azure destination.

## Establish a Connection and Define URLs

In [1]:
import sys
sys.path.append('..')
import mmAuthorization

import requests
import json, os, pprint
import base64

In [ ]:
host_name= "localhost"
port = "8080"

host_url="http://" + host_name + ":" + port
destination_url = host_url + "/modelPublish/destinations/"
modelrepo_url = host_url + "/modelRepository/models/"
publishmodel_url = host_url + "/modelPublish/models"
domains_url = host_url + "/credentials/domains"
print(host_url)

## Get an Authorization Token and Define Headers

In [ ]:
mm_auth = mmAuthorization.mmAuthorization("myAuth")

admin_userId = '<SAS_user_admin_ID>'
user_passwd =  '<SAS_user_password>'

admin_auth_token = mm_auth.get_auth_token(host_url, admin_userId, user_passwd)

credential_admin_headers = {
    mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + admin_auth_token
}

credential_domain_headers = {
    "If-Match":"false",
    "Content-Type":"application/json",
    mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + admin_auth_token
}

credential_user_headers = {
    "If-Match":"false",
    "Content-Type":"application/json",
    mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + admin_auth_token
}

destination_azure_headers = {
    "If-Match":"false",
    "Content-Type":"application/vnd.sas.models.publishing.destination.azure+json",
    mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + admin_auth_token
}

print(admin_auth_token)

## Create a Credential Domain

In [ ]:
domain_name = '<Domain_Name>'
description = '<Domain_Description>'

my_domain_url = domains_url + "/" + domain_name
domain_attrs = {
    "id": domain_name,
    "type": "base64",
    "description": description
}

domain = requests.put(my_domain_url, 
                       data=json.dumps(domain_attrs), headers=credential_domain_headers)

print(domain)
pprint.pprint(domain.json())

## Create User Credentials

In [ ]:
domain_name = '<Domain_Name>'
my_domain_url = domains_url + "/" + domain_name

user_credential_name = admin_userId
my_credential_url = my_domain_url + "/users/" + user_credential_name

user_id = '<Azure_Container_Registry_User_Name>'
password = '<Azure_Container_Registry_Password>'

encoded_user_id = str(base64.b64encode(user_id.encode("utf-8")), "utf-8")
encoded_password = str(base64.b64encode(password.encode("utf-8")), "utf-8")

serverAppId = '<Active_Directory_Server_Application_ID>' 
serverAppPassword = '<Active_Directory_Server_Application_Secret>'

encodedAppPassword = str(base64.b64encode(serverAppPassword.encode("utf-8")), "utf-8")

credential_attrs = {
    "domainId":domain_name,
    "identityType":"user",
    "identityId":user_credential_name,
    "domainType":"base64",
    "properties":{"dockerRegistryUserId":encoded_user_id,
                  "azureAppId":serverAppId},
    "secrets":{"dockerRegistryPasswd":encoded_password,
               "azureAppPasswd":encodedAppPassword}   
}

credential = requests.put(my_credential_url, 
                       data=json.dumps(credential_attrs), headers=credential_user_headers)

print(credential)
pprint.pprint(credential.json())

## Create an Azure Destination

In [ ]:
dest_name = '<Destination_Name>'
domainName = '<Domain_Name>'
baseRepoUrl = '<Azure_Container_Registry_Base_URL>'

tenantId = '<Azure_Tenant_ID>'    
subscriptionId = '<Azure_Subscription_ID>'
resourceGroupName = '<Azure_Resource_Group_Name>'
k8sClusterName = '<Azure_Kubernetes_Service_Cluster_Name>'
externalIP = '<External_IP>' # Public IP address of one of the VMs associated with the cluster.


destination_attrs = {
    'name': dest_name,
    'destinationType': 'azure',
    'properties': [{'name': "credDomainId", "value": domainName},
                   {"name": "kwsImageUrl", "value": "myserver.com/kws:latest"},
                   {"name": "baseRepoUrl", "value": baseRepoUrl},
                   {'name': "tenantId", "value": tenantId},
                   {'name': "subscriptionId", "value": subscriptionId},
                   {'name': "resourceGroupName", "value": resourceGroupName},
                   {'name': "kubernetesCluster", "value": k8sClusterName},
                   {'name': "externalIP", "value": externalIP}]}

destination = requests.post(destination_url, 
                       data=json.dumps(destination_attrs), headers=destination_azure_headers)

print(destination)
pprint.pprint(destination.json())